In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import gamma
from matplotlib import pyplot as plt
from typing import Tuple, List, Callable, Any
from file_util import *

In [ ]:
# General settings
num_runs = 1
starting_seed = 0
seed_multiplier = 100

# Validator settings
num_nodes = 31 
num_consensus = 2000
base_time_limit = 10000
node_processing_distribution = "exp"
node_processing_parameters = [3]
consensus_protocol = "HS"

## Fault settings
num_faults = 0
fault_type = "UR"
fault_parameters = []

# Network settings
## Switch settings
switch_processing_distribution = "exp"
switch_processing_parameters = [0]
message_channel_success_rate = 1

network_type = "df"
network_parameters = []

In [ ]:
# More utility methods for analysis
def get_minima(series: pd.Series):
    return series[(series < series.shift(1)) & (series < series.shift(-1))].iloc[0]

def get_minima_index(series: pd.Series):
    return series[(series < series.shift(1)) & (series < series.shift(-1))].index[0]

In [ ]:

def get_topology_data(topo: str, protocol: str, name: str, topo_params: List[int]=None, dir=DEFAULT_RESULTS_DIRECTORY):
    return get_fn_data(lambda name: name.get_switch_params()[0], lambda json: json[TOTAL_TIME_KEY], name,
                       num_nodes=num_nodes, base_time_limit=base_time_limit, 
                       topology=topo, protocol=protocol, num_faults=num_faults, node_dist=node_processing_distribution, 
                       topology_params=topo_params, dir=dir)

In [ ]:
ibft_series_8_4_one = get_topology_data("fc", "ibft", "ibft_fc_8_4_1p", topo_params=[8, 4])
hs_series_8_4_one = get_topology_data("fc", "hs", "hs_fc_8_4_1p", topo_params=[8, 4])
ibft_series_8_4_two = get_topology_data("fc", "ibft", "ibft_fc_8_4_2p", topo_params=[8, 4], dir="results/2_simul_programs/")
hs_series_8_4_two = get_topology_data("fc", "hs", "hs_fc_8_4_2p", topo_params=[8, 4], dir="results/2_simul_programs")
ibft_series_8_4_three = get_topology_data("fc", "ibft", "ibft_fc_8_4_3p", topo_params=[8, 4], dir="results/3_simul_programs/")
hs_series_8_4_three = get_topology_data("fc", "hs", "hs_fc_8_4_3p", topo_params=[8, 4], dir="results/3_simul_programs")


In [ ]:

topo_df_one = pd.DataFrame([ibft_series_8_4_one, hs_series_8_4_one]).transpose()
topo_df_two = pd.DataFrame([ibft_series_8_4_two, hs_series_8_4_two]).transpose()
topo_df_three = pd.DataFrame([ibft_series_8_4_three, hs_series_8_4_three]).transpose()

topo_df_one.loc[5:].plot(style=".-", grid=True, title="One programs, Folded clos, n=31, half radix = 4")
topo_df_two.loc[5:].plot(style=".-", grid=True, title="Two programs, Folded clos, n=31, half radix = 4")
topo_df_three.loc[5:].plot(style=".-", grid=True, title="Three programs, Folded clos, n=31, half_radix = 4")

In [ ]:
hs_df = pd.DataFrame([hs_series_8_4_one, hs_series_8_4_two, hs_series_8_4_three]).transpose().dropna()
ibft_df = pd.DataFrame([ibft_series_8_4_one, ibft_series_8_4_two, ibft_series_8_4_three]).transpose().dropna()


def ibft_fc_model(n, k, mu, _lambda):
    min_time = (2 * n + 1) / mu
    model_time = 2 * k * (2 * n - 1) / _lambda
    return max(min_time, model_time) 
def hs_pred_fn(n, mu, _lambda):
    f = (n - 1) // 3
    pred_1 = (n - 1) * 8 / _lambda
    pred_2 = (4 * (n - f)) / mu + 3 * (n) / _lambda + (n / 2 + 4.65) / _lambda 
    pred_3 = (4 * (n - f)) / mu + 3 * (f + 1) / mu + (n / 2 + 4.65) / _lambda
    # first term is processing delay per round, n - f at leader and 1 elsewhere
    # second term is remainder time that the leader clears up redundant extra messages vs time the first next-phase message takes to enter. Happens for 3 phases.
    # Last term is the average time taken for the final message to reach the desired node, with n/2 being half the nodes and 5.23 being the expected hop count
    return max(pred_1, pred_2, pred_3)


m = 2
ibft_df["pred"] = ibft_df.index.map(lambda rate : ibft_fc_model(31, 4, 3 / m, rate / m))


hs_df.loc[5:].plot(style=".-", grid=True, title="HotStuff, FoldedClos, n=31, half radix=4", xlim=[0, max(hs_df.index)], ylim=[0, max(hs_df.loc[5:].max())])
plt.show()
hs_df.loc[5:].plot(style=".-", grid=True, title="HotStuff, FoldedClos, n=31, half radix=4")
plt.show()
ibft_df.loc[5:].plot(style=".-", grid=True, title="IBFT, FoldedClos, n=31, half radix=4")
plt.show()
ibft_df.loc[5:].plot(style=".-", grid=True, title="IBFT, FoldedClos, n=31, half radix=4", xlim=[0, max(ibft_df.index)], ylim=[0, max(ibft_df.loc[5:].max())])